In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [3]:
train_df = pd.read_csv('./new_train.csv', parse_dates=["startdate"])
test_df = pd.read_csv('./new_test.csv', parse_dates=["startdate"])

In [4]:
exclude_cols = ['index', 'startdate']
temporal_attrs = ['year', 'month', 'day_of_year', 'season', 'day_of_year_sin', 'day_of_year_cos','month_sin', 'month_cos', 'season_sin', 'season_cos']
loc_attrs = ['lat', 'lon', 'loc_group']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in temporal_attrs and c not in loc_attrs and c not in target and c not in embedding_attrs]
print("Main features:", len(main_attrs))

Main features: 240


In [17]:
best_cols = ['contest-wind-h500-14d__wind-hgt-500', 'contest-slp-14d__slp',
       'nmme-tmp2m-34w__ccsm3', 'elevation__elevation', 'lon',
       'contest-prwtr-eatm-14d__prwtr', 'lat', 'climateregions__climateregion',
       'contest-pres-sfc-gauss-14d__pres', 'season_sin', 'day_of_year_sin',
       'contest-precip-14d__precip',
       'contest-wind-uwnd-250-14d__wind-uwnd-250', 'nmme-prate-34w__cfsv2',
       'nmme-prate-34w__nasa', 'nmme-prate-56w__gfdlflora',
       'wind-uwnd-250-2010-7', 'contest-wind-vwnd-925-14d__wind-vwnd-925',
       'nmme-prate-34w__nmmemean', 'nmme0-prate-34w__ccsm30',
       'contest-wind-h850-14d__wind-hgt-850',
       'contest-wind-uwnd-925-14d__wind-uwnd-925', 'nmme0-prate-56w__cfsv20',
       'nmme-prate-34w__cancm3', 'contest-rhum-sig995-14d__rhum',
       'nmme-prate-34w__gfdlflorb', 'wind-hgt-850-2010-4',
       'contest-wind-vwnd-250-14d__wind-vwnd-250', 'wind-hgt-100-2010-2',
       'wind-uwnd-250-2010-18', 'wind-hgt-10-2010-5', 'wind-uwnd-250-2010-15',
       'wind-uwnd-250-2010-4', 'nmme0-prate-56w__nasa0',
       'nmme0-prate-34w__cfsv20', 'wind-vwnd-250-2010-10',
       'contest-wind-h10-14d__wind-hgt-10', 'wind-uwnd-925-2010-15',
       'wind-vwnd-250-2010-3', 'nmme-prate-34w__cancm4', 'sst-2010-4',
       'nmme0-prate-56w__ccsm30', 'wind-uwnd-250-2010-16',
       'nmme0-prate-34w__gfdl0', 'nmme0-prate-56w__cancm40', 'sst-2010-1',
       'sst-2010-3', 'wind-uwnd-250-2010-14', 'nmme0-prate-34w__gfdlflora0',
       'nmme-prate-34w__gfdl', 'wind-hgt-850-2010-9', 'wind-vwnd-250-2010-1',
       'sst-2010-5', 'cancm30', 'nmme-prate-34w__ccsm4',
       'nmme0-prate-34w__nasa0', 'wind-hgt-500-2010-9',
       'nmme0-prate-34w__cancm30', 'wind-vwnd-250-2010-13']

# important_cols = ['contest-wind-h500-14d__wind-hgt-500']

In [18]:
train_df[['startdate'] + best_cols + target].to_csv('sub_train.csv', index=False)
test_df[['startdate'] + best_cols + ['index']].to_csv('sub_test.csv', index=False)